# The Core module

The Core module implements a framework for simultaneous control of many networked devices. Core classes can also describe experiments, which can be run with the ARTIQ module and optimized with the Pipeline module - after writing an experiment in a Hub class, just attach it to the optimization pipeline. The hierarchy is as follows:

The lowest-level object is a Knob, a class representing a degree of freedom in your experiment, like a laser frequency or a voltage. Knobs are attached to Devices, a class which translates device drivers to standard EMERGENT syntax. The Hub class unites multiple Devices to optimize experiments defined within it. Multiple hubs can be overseen by the top-level Core class.

## Step 1: Preparing the directories
The entire collection of knobs, devices, and higher-level objects is called a "network", and is defined inside a directory in the emergent/networks folder. Let's create an empty network called "test":

In [2]:
import os
import json
with open('../../config.json', 'r') as file:
    path = json.load(file)['path']
%cd $path
%run utilities/new test

/Users/robbiefasano/emergent/emergent


You should see a new folder called test in the emergent/networks/ directory. 

## Step 2: Writing device drivers
Next, we're going to define the core building blocks of an EMERGENT network: a Thing, a Hub, and a network declaration file. Each of these is typically its own .py file; here, we'll define them within the notebook and save them to files as we go using IPython's magic %%writefile command.

First, let's define a Thing: a driver
for some device in our experiment. Although Things are just standard Python classes which can contain any methods
you want, they should always have methods called \_actuate() and \_connect().

<div class="alert alert-block alert-info">
<b>_actuate():</b> takes a state dictionary as an argument. Overload this method according to the manufacturer API to 
send the state specified by the dictionary to the device to do something useful in the lab.
<br>
<b>_connect():</b> establishes a connection with the device and returns 1 if successful. Overload this method with 
the specific connection protocol required by the device, e.g. sending a start packet over TCP/IP.
</div>


Things also have standardized arguments which should be passed into the super().__init__() method as well:
<div class="alert alert-block alert-info">
<b>name:</b> the name which will be displayed in the experimental state dictionary and the GUI.
<br>
<b>params:</b> a dictionary containing any parameters you might want to pass to the device, like a serial number or 
analog input range.
<br>
<b>parent:</b> the Hub to which this Thing will be attached.
</div>

In [3]:
%%writefile $path/networks/test/things/test_thing.py
from emergent.core import Thing

class TestThing(Thing):
    ''' Thing driver for the virtual network in the 'basic' example. '''
    def __init__(self, name, params = {}, parent = None):
        ''' Register with the network and create two Knobs, 'X' and 'Y'. '''
        super().__init__(name, parent, params = params)
        
        for knob_name in ['X', 'Y']:
            self.add_knob(knob_name)

    def _actuate(self, state):
        ''' Usually this method would change a physical state, but for our
            virtual network we only print the argument (note: the virtual state
            is updated within the public calling method Thing.actuate()).'''
        return

    def _connect(self):
        return 1

Overwriting /Users/robbiefasano/emergent/emergent/networks/test/things/test_thing.py


## Step 3: Writing experiments
Now we will construct a Hub: a virtual construct which commands multiple things and measures some attached signal. 
The __init__ method should take the following arguments and pass them into the super().__init__ method:
<div class="alert alert-block alert-info">
<b>name:</b> the name which will be displayed in the experimental state dictionary and the GUI.
<br>
<b>params:</b> a dictionary containing any parameters you might want to pass to the hub.
<br>
<b>network:</b> the local cluster of Hubs to associate this instance with.
<br>
<b>addr:</b> the IP address of the PC where you want this Hub to run. The Hub will only be constructed if the 
PC has a network card matching this address. This allows Hubs across multiple decentralized PCs to be declared in
a single network.py file and selectively constructed depending on which PC is running the network.initialize() method.
</div>

In EMERGENT, experiments are written as methods of Hub classes with a standard call signature:
<div class="alert alert-block alert-info">
<b>state:</b> a dictionary specifying the state for which we want to run the experiment. For example, if we want
to measure a signal with a Thing named 'thing' set to coordinates X=1 and Y=2, we would pass {'thing': {'X':1, 'Y':2}}.
<br>
<b>params:</b> a dictionary containing any parameters you might want to pass to the experiment, e.g. an averaging
time.
</div>

The method is tagged with the @experiment decorator, which tells EMERGENT to treat it differently than a normal method: all tagged methods appear in spin-boxes in the GUI.

In [4]:
%%writefile $path/networks/test/hubs/test_hub.py 
import numpy as np
import time
from emergent.core import Hub
from emergent.utilities.decorators import experiment, error

class TestHub(Hub):
    def __init__(self, name, params = {}, addr=None, core=None):
        super().__init__(name, addr=addr, core = core, params = params)
        self.options['Hello'] = self.hello
        
    def hello(self, name='world'):
        print('Hello', name)
        
    @experiment
    def gaussian(self, state, params = {'sigma_x': 0.3, 'sigma_y': 0.8, 'x0': 0.3, 'y0': 0.6, 'noise':0}):
        self.actuate(state)
        x=self.state['thing']['X']
        y=self.state['thing']['Y']
        x0 = params['x0']
        y0 = params['y0']
        sigma_x = params['sigma_x']
        sigma_y = params['sigma_y']
        power =  np.exp(-(x-x0)**2/sigma_x**2)*np.exp(-(y-y0)**2/sigma_y**2) + np.random.normal(0, params['noise'])

        return -power
    
    @error 
    def error_function(self, state, params={'setpoint': 1}):
        self.actuate(state)
        return self.state['thing']['X'] - params['setpoint']
    

Overwriting /Users/robbiefasano/emergent/emergent/networks/test/hubs/test_hub.py


## Step 4: Initializing the network
The last step is to declare our objects in a network.py file, which should contain only a method called initialize() which is structured as follows:

In [5]:
%%writefile $path/networks/test/network.py

from emergent.networks.test.hubs.test_hub import TestHub
from emergent.networks.test.things.test_thing import TestThing
from emergent.artiq.emergent_sequencer import Sequencer

def initialize(core, params = {}):
    core.add_params(params)          # add the passed params to the network

    hub = TestHub(name='hub', core=core, addr='127.0.0.1')
    thing = TestThing('thing', params={}, parent=hub)
    
    core.add_hub(hub)

Overwriting /Users/robbiefasano/emergent/emergent/networks/test/network.py


Now, launch the network!

In [6]:
%run master test --port 6000

 * Serving Flask app "emergent.API.API" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


# State representation and actuation

You can access the objects and state from the command-line using the global "core" variable, as in the following example:

In [7]:
hub = core.hubs['hub']
thing = hub.children['thing']
X = thing.children['X']

print(hub.state)

DataDict([('thing', {'X': 1.0, 'Y': 2.0})])


To change a device state, you can pass a state dict into the actuate() method of either the Thing or Hub. The following commands are equivalent:

In [8]:
thing.actuate({'X': 1, 'Y': 2})
hub.actuate({'thing': {'X': 1, 'Y': 2}})
print('New state:', hub.state)

New state: DataDict([('thing', {'X': 1, 'Y': 2})])


The hub also possesses a range attribute that defines the bounds in optimization processes.

In [13]:
hub.range

DataDict([('thing',
           {'X': {'min': 0.0, 'max': 1.0}, 'Y': {'min': 0.0, 'max': 1.0}})])